In [7]:
import pandas as pd
import numpy as np
import si4automl

df = pd.read_csv("cancer.tsv", sep="\t")
print(df.shape)
y = df["lpsa"]
X = df.drop(["lpsa", "Unnamed: 0", "train"], axis=1)
print(labels := X.columns)
X = X.values

print(y.shape, X.shape)
X = (X - np.mean(X, axis=0)) / np.std(X, axis=0)
print(X.shape)

y = (y - np.mean(y)) / np.std(y)


def option():
    X, y = si4automl.initialize_dataset()
    y = si4automl.mean_value_imputation(X, y)

    O = si4automl.soft_ipod(X, y, 0.015)
    X, y = si4automl.remove_outliers(X, y, O)

    M = si4automl.marginal_screening(X, y, 5)
    X = si4automl.extract_features(X, M)

    M1 = si4automl.stepwise_feature_selection(X, y, 3)
    M2 = si4automl.lasso(X, y, 0.08)
    M = si4automl.union(M1, M2)
    return si4automl.construct_pipelines(output=M)

manager = option()
M, O = manager(X, y)
print(M, O)
print(labels[M])
print()

M, results = manager.inference(X, y, retain_result=True)

print([r.p_value for r in results])
print(manager.inference(X, y, inference_mode="over_conditioning")[1])
print([r.naive_p_value() for r in results])

(97, 11)
Index(['lcavol', 'lweight', 'age', 'lbph', 'svi', 'lcp', 'gleason', 'pgg45'], dtype='object')
(97,) (97, 8)
(97, 8)
[0, 1, 4, 7] [38]
Index(['lcavol', 'lweight', 'svi', 'pgg45'], dtype='object')

[0.000150491577944889, 0.01847881174944177, 0.06980997352081564, 0.7369299275116837]
[0.000150491577944889, 0.20100807283242783, 0.05814979771097273, 0.7304913247221283]
[2.21358103140832e-11, 5.268747028860415e-05, 0.0013070871456503875, 0.3897670711150762]
